In [11]:
import os
import pandas as pd
import csv
import math
import numpy as np
from ipywidgets import Dropdown
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr


In [12]:
'''folder_path = 'SaliencyData/Task1'

def load_data(file):
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path, delimiter=';', usecols=['GazeTime', 'Region', 'Target', 'PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ', 'LightIntensity', 'Velocity'])
    data = data[~((data['PosX'] == 0) & (data['PosY'] == 0) & (data['PosZ'] == 0))

    if len(data) <= 1:  # The data contains only a header
        return None

    # Get the unique regions
    regions = data['Region'].unique()

    # Skip the first region
    for region in regions[1:]:
        # Get the index of the first row of the current region
        first_row_index = data[data['Region'] == region].index[0]
        # Drop the first row of the current region
        data = data.drop(first_row_index)

    # Normalize position data using Min-Max normalization
    pos_columns = ['PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ', 'LightIntensity', 'Velocity']
    for column in pos_columns:
        data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())

    return data

# Create a list to store the dataframes
dataframes = [df for df in (load_data(file) for file in os.listdir(folder_path) if file.endswith('.csv')) if df is not None]
os.listdir(folder_path)
dataframes[0]'''

"folder_path = 'SaliencyData/Task1'\n\ndef load_data(file):\n    file_path = os.path.join(folder_path, file)\n    data = pd.read_csv(file_path, delimiter=';', usecols=['GazeTime', 'Region', 'Target', 'PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ', 'LightIntensity', 'Velocity'])\n    data = data[~((data['PosX'] == 0) & (data['PosY'] == 0) & (data['PosZ'] == 0))\n\n    if len(data) <= 1:  # The data contains only a header\n        return None\n\n    # Get the unique regions\n    regions = data['Region'].unique()\n\n    # Skip the first region\n    for region in regions[1:]:\n        # Get the index of the first row of the current region\n        first_row_index = data[data['Region'] == region].index[0]\n        # Drop the first row of the current region\n        data = data.drop(first_row_index)\n\n    # Normalize position data using Min-Max normalization\n    pos_columns = ['PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ', 'LightIntensity', 'Velocity']\

In [18]:
def load_data(file):
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path, delimiter=';', usecols=['GazeTime', 'Region', 'Target', 'PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ', 'LightIntensity', 'Velocity'])
    data = data[~((data['PosX'] == 0) & (data['PosY'] == 0) & (data['PosZ'] == 0))]

    if len(data) <= 1:  # The data contains only a header
        return None

    # Normalize position data using Min-Max normalization
    pos_columns = ['PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ', 'LightIntensity', 'Velocity']
    for column in pos_columns:
        data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())

    return data

folder_path = 'SaliencyData/Task1'

# List directory and sort files by their numerical value after '_'
files = sorted(os.listdir(folder_path), key=lambda x: int(x.split('_')[1].split('.csv')[0]))

# Create a list to store the dataframes
dataframes = [df for df in (load_data(file) for file in files if file.endswith('.csv')) if df is not None]



In [19]:
def correct_regions(df):
    # Define valid targets for each region
    valid_targets = {
        1: ['Size(Min)', 'Size(Low)', 'Size(Mid)', 'Size(Max)','None'],
        2: ['Transparency(Min)', 'Transparency(Low)', 'Transparency(Mid)', 'Transparency(Max)','None'],
        3: ['Red', 'White', 'Black', 'Blue','None']
    }
    
    # Define a function to find the correct region for a given target
    def find_correct_region(row):
        if row['Target'] == 'None':
            return row['Region']  # Keep the current region if the target is 'None'
        for region, targets in valid_targets.items():
            if row['Target'] in targets:
                return region
        return row['Region']  # Keep the current region if the target does not belong to any defined region

    # Apply the function to each row in the DataFrame
    df['Corrected_Region'] = df.apply(find_correct_region, axis=1)
    # Replace 'Region' column with 'Corrected_Region', then drop 'Corrected_Region'
    df['Region'] = df['Corrected_Region']
    df.drop('Corrected_Region', axis=1, inplace=True)
    
    return df

# Apply the function to each DataFrame in your list
dataframes = [correct_regions(df) for df in dataframes]


In [20]:
def target_counts(df):
    # Filter out rows where Target is 'None'
    df_filtered = df[df['Target'] != 'None']
    # Count the number of times each 'Target' was looked at
    target_counts = df_filtered['Target'].value_counts().reset_index()
    target_counts.columns = ['Target', 'Count']
    return target_counts

# Use the target_counts function on each DataFrame in your list
all_target_counts = pd.concat([target_counts(df) for df in dataframes])

# Sum the counts for each 'Target' across all DataFrames
total_target_counts = all_target_counts.groupby('Target').sum().reset_index()

print(total_target_counts)


               Target  Count
0               Black     22
1                Blue      1
2                 Red     20
3           Size(Low)     12
4           Size(Max)     85
5           Size(Mid)     26
6           Size(Min)      6
7   Transparency(Low)     20
8   Transparency(Max)     22
9   Transparency(Mid)     23
10  Transparency(Min)     21
11              White     21


In [21]:
def ProximtyVsGazeTime(df):

    # Calculate the Euclidean distance between the object and player positions
    df['Proximity'] = np.sqrt((df['PosX'] - df['PlayerPosX'])**2 + (df['PosY'] - df['PlayerPosY'])**2 + (df['PosZ'] - df['PlayerPosZ'])**2)

    # Create a scatter plot for GazeTime vs Proximity
    plt.figure(figsize=(10, 5))
    plt.scatter(df['Proximity'], df['GazeTime'])
    plt.title('GazeTime vs Proximity')
    plt.xlabel('Proximity')
    plt.ylabel('GazeTime')
    plt.show()

    # Calculate correlation between GazeTime and Proximity
    correlation, _ = pearsonr(df['GazeTime'], df['Proximity'])
    print('Pearson correlation: %.3f' % correlation)

    



In [22]:
def sum(arr):
    ret = 0
    for i in arr:
        ret += i
    return ret

def PercFixInside(dataframe):
    def Compute(region, interest):
        ret = pd.DataFrame()
        target_names = dataframe[dataframe['Region'] == region]['Target'].unique()
        grouped_data = dataframe[dataframe['Region'] == region].groupby('Target')
        gaze = [0] * len(target_names)  # Initialize list with zeros
        perc = [0] * len(target_names)
        ret['Target'] = target_names
        
        for j, target in enumerate(target_names):
            gaze[j] = grouped_data.get_group(target)[interest].sum()
        tot = sum(gaze)

        for i, g in enumerate(gaze):
            perc[i] = (g / tot) * 100 if tot > 0 else 0

        ret['GazeTime'] = gaze
        ret['Percentage'] = perc
        return ret

    regions = dataframe['Region'].unique()

    # Initialize a list to store the DataFrames for each region
    results = []

    for region in regions:
        newdata = Compute(region, 'GazeTime')
        newdata['Region'] = region  # Add a column for the region
        results.append(newdata)

    # Concatenate all the result DataFrames into one
    final_df = pd.concat(results, ignore_index=True)

    return final_df

PercFixInside(dataframes[10])

,Target,GazeTime,Percentage,Region
0,Size(Max),0.083910,4.270790,1
1,None,1.532658,78.008097,1
2,Size(Mid),0.348174,17.721113,1
3,Transparency(Min),0.097412,5.033351,2
4,None,1.629213,84.182573,2
5,Transparency(Max),0.208708,10.784076,2
6,None,1.740877,100.000000,3


In [23]:
def NFix(dataframe, threshold, ShowPercentage = False):
    if(threshold < 0):
        threshold = abs(threshold)
    nfix = {}  # Initialize the NFix counter as a dictionary
    counter = {}  # Initialize the counter as a dictionary
    pos = {}  # Variables to track previous position for each target

    for _, row in dataframe.iterrows():
        region = row['Region']
        target = row['Target']
        if region not in counter:
            counter[region] = {}
            nfix[region] = {}
        if target not in counter[region]:
            counter[region][target] = 0
            nfix[region][target] = 0
            pos[region, target] = (None, None)

        counter[region][target] += 1
        pos_x_new, pos_y_new = float(row['PosX']), float(row['PosY'])  # Get the X and Y positions

        # Check if a fixation occurred by comparing the current position with the previous position
        if pos[region, target][0] is not None and pos[region, target][1] is not None:
            pos_diff = abs(pos[region, target][0] - pos_x_new) + abs(pos[region, target][1] - pos_y_new)
            if pos_diff > threshold:
                nfix[region][target] += 1  # Increment the NFix counter for the target

        pos[region, target] = (pos_x_new, pos_y_new)  # Update the previous position for the target

    if ShowPercentage:
        for region in nfix:
            for target in nfix[region]:
                nfix[region][target] = (nfix[region][target] / counter[region][target]) * 100 if counter[region][target] > 0 else 0

    # Convert nested dictionary to DataFrame
    nfix_df = pd.DataFrame([(reg, tar, val) for reg, tar_val in nfix.items() for tar, val in tar_val.items()], 
                           columns=['Region', 'Target', 'NFix'])

    return nfix_df

In [24]:
def ConvergTime(dataframe):
    # Group the data by region and then by target
    dataframe = dataframe[dataframe['Target'] != 'None']
    grouped_data = dataframe.groupby(['Region', 'Target'])

    # Initialize an empty DataFrame to store the results
    results = pd.DataFrame(columns=['Region', 'Target', 'ConvergenceTime'])

    # Iterate over each group (region and target)
    for (region, target), group in grouped_data:
        # Calculate the average gaze time for the current group (target within the region)
        avg_gaze_time = group['GazeTime'].mean()
        
        # Append the result to the results DataFrame
        results = pd.concat([results, pd.DataFrame({'Region': [region], 'Target': [target], 'ConvergenceTime': [avg_gaze_time]})], ignore_index=True)

    return results

In [25]:
def calculate_proximity(df):
    # Calculate the Euclidean distance
    df['Proximity'] = np.sqrt((df['PosX'] - df['PlayerPosX'])**2 + (df['PosY'] - df['PlayerPosY'])**2 + (df['PosZ'] - df['PlayerPosZ'])**2)
    # Remove rows with 'None' Target
    df = df[df['Target'] != 'None']
    # Group by 'Target' and calculate mean distance
    df = df.groupby('Target')['Proximity'].mean().reset_index()
    return df




In [26]:
calculate_proximity(dataframes[10])

,Target,Proximity
0,Size(Max),0.915822
1,Size(Mid),0.803662
2,Transparency(Max),1.144654
3,Transparency(Min),0.989266


In [27]:
# Step 1: Apply PercFixInside to each DataFrame in dataframes and accumulate the results


perc_results = [PercFixInside(df) for df in dataframes]

# Step 2: Concatenate the resulting DataFrames into one
all_perc = pd.concat(perc_results)

# Step 3: Group by 'Region' and 'Target' and calculate the mean
avg_perc = all_perc.groupby(['Region', 'Target']).mean().reset_index()

# This will now be a DataFrame with average 'GazeTime' and 'Percentage' for each 'Target' in each 'Region'
for i,j in enumerate(avg_perc['Percentage']):
    #print(str(i)+" : "+ str(j))
    avg_perc.loc[2,i] = j/len(dataframes)
avg_perc

,Region,Target,GazeTime,Percentage,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,None,1.737019,63.203026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Size(Low),0.259207,12.094703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Size(Max),1.074800,45.852485,2.257251,0.431954,1.637589,0.491965,0.711313,2.570713,0.902405,0.449974,0.784976,0.577037,0.523522,0.280988,2.440483,0.980448,1.280447
3,1,Size(Mid),0.421225,13.775023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Size(Min),0.436598,19.916765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,None,1.592143,71.979965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,Transparency(Low),0.767658,25.267348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2,Transparency(Max),0.242981,12.599280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,Transparency(Mid),0.391355,21.979339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2,Transparency(Min),0.352426,16.157024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Step 1: Apply PercFixInside to each DataFrame in dataframes and accumulate the results
perc_results = [PercFixInside(df) for df in dataframes]

# Step 2: Concatenate the resulting DataFrames into one
all_perc = pd.concat(perc_results)

# Step 3: Group by 'Region' and 'Target' and calculate the sum of GazeTime
total_gaze = all_perc.groupby(['Region', 'Target'])['GazeTime'].sum().reset_index()

# Step 4: Calculate the total gaze time in each region
total_gaze_region = total_gaze.groupby('Region')['GazeTime'].sum()

# Step 5: Calculate the percentage of gaze time that each target represents in its region
total_gaze['Percentage'] = total_gaze.apply(lambda row: (row['GazeTime'] / total_gaze_region[row['Region']]) * 100, axis=1)

# This will now be a DataFrame with total 'GazeTime' and 'Percentage' for each 'Target' in each 'Region'
total_gaze


,Region,Target,GazeTime,Percentage
0,1,None,45.162502,61.660145
1,1,Size(Low),1.296037,1.769473
2,1,Size(Max),20.421202,27.880968
3,1,Size(Mid),5.054699,6.901156
4,1,Size(Min),1.309795,1.788257
5,2,None,42.987870,70.815179
6,2,Transparency(Low),8.444235,13.910436
7,2,Transparency(Max),2.186827,3.602424
8,2,Transparency(Mid),3.913553,6.446911
9,2,Transparency(Min),3.171831,5.225050


In [29]:
def calculate_avg_perc(dataframes):
    # Apply PercFixInside to each DataFrame in dataframes, filter out 'None' target and accumulate the results
    perc_results = []
    for df in dataframes:
        if df[df['Target'] != 'None'].empty:  # Check if filtered dataframe is empty
            continue  # Skip to next dataframe if it's empty
        perc_results.append(PercFixInside(df[df['Target'] != 'None']))
    
    # Concatenate the resulting DataFrames into one
    all_perc = pd.concat(perc_results)

    # Group by 'Region' and 'Target' and calculate the mean
    avg_perc = all_perc.groupby(['Region', 'Target']).mean().reset_index()

    # This will now be a DataFrame with average 'GazeTime' and 'Percentage' for each 'Target' in each 'Region'
    return avg_perc
calculate_avg_perc(dataframes)

,Region,Target,GazeTime,Percentage
0,1,Size(Low),0.259207,49.634623
1,1,Size(Max),1.074800,75.079893
2,1,Size(Mid),0.421225,33.252387
3,1,Size(Min),0.436598,42.093423
4,2,Transparency(Low),0.767658,51.231480
5,2,Transparency(Max),0.242981,33.914342
6,2,Transparency(Mid),0.391355,46.644967
7,2,Transparency(Min),0.352426,40.530552
8,3,Black,0.286808,55.027354
9,3,Blue,0.153389,15.309233


In [30]:
# Step 1: Apply NFix to each DataFrame in dataframes and accumulate the results
nfix_results = [NFix(df, 0.05, False) for df in dataframes]

# Step 2: Concatenate the resulting DataFrames into one
all_nfix = pd.concat(nfix_results)

# Step 3: Group by 'Region' and 'Target' and calculate the mean
avg_nfix = all_nfix.groupby(['Region', 'Target']).mean().reset_index()

# This will now be a DataFrame with average 'Count' and 'Percentage' for each 'Target' in each 'Region'
avg_nfix

,Region,Target,NFix
0,1,None,6.423077
1,1,Size(Low),1.000000
2,1,Size(Max),2.789474
3,1,Size(Mid),1.000000
4,1,Size(Min),0.333333
5,2,None,8.740741
6,2,Transparency(Low),0.727273
7,2,Transparency(Max),1.444444
8,2,Transparency(Mid),1.300000
9,2,Transparency(Min),1.111111


In [33]:
def overall_average_distance(dataframes):
    # Create a list to store the distance dataframes
    distance_dfs = []

    # Loop through each dataframe in dataframes
    for df in dataframes:
        # Calculate the distance for each Target in the dataframe
        distance_df = calculate_proximity(df)
        # Append the resulting dataframe to the list
        distance_dfs.append(distance_df)

    # Concatenate all distance dataframes into a single dataframe
    all_data = pd.concat(distance_dfs)

    # Group by Target and calculate the average distance
    all_data = all_data.groupby('Target')['Proximity'].mean().reset_index()

    return all_data

# usage
overall_avg_distances = overall_average_distance(dataframes)
overall_avg_distances

,Target,Proximity
0,Black,0.983988
1,Blue,0.981388
2,Red,1.223024
3,Size(Low),0.885860
4,Size(Max),0.898982
5,Size(Mid),0.911903
6,Size(Min),0.957918
7,Transparency(Low),0.776292
8,Transparency(Max),0.797452
9,Transparency(Mid),0.686023


In [34]:
def overall_average_convergence_time(dataframes):
    # Create a list to store the convergence time dataframes
    convergence_time_dfs = []

    # Loop through each dataframe in dataframes
    for df in dataframes:
        # Calculate the convergence time for each 'Region' and 'Target' in the dataframe
        convergence_time_df = ConvergTime(df)
        # Append the resulting dataframe to the list
        convergence_time_dfs.append(convergence_time_df)

    # Concatenate all convergence time dataframes into a single dataframe
    all_data = pd.concat(convergence_time_dfs)

    # Group by 'Region' and 'Target' and calculate the average convergence time
    all_data = all_data.groupby(['Region', 'Target'])['ConvergenceTime'].mean().reset_index()

    return all_data

# usage
overall_avg_convergence_times = overall_average_convergence_time(dataframes)
overall_avg_convergence_times

,Region,Target,ConvergenceTime
0,1,Size(Low),0.109187
1,1,Size(Max),0.423422
2,1,Size(Mid),0.281024
3,1,Size(Min),0.218299
4,2,Transparency(Low),0.625256
5,2,Transparency(Max),0.103947
6,2,Transparency(Mid),0.196282
7,2,Transparency(Min),0.194338
8,3,Black,0.147739
9,3,Blue,0.153389


In [35]:
import numpy as np

def calculate_eccentricity(dataframe):
    # Get vectors from the eye to the object and from the eye straight ahead
    dataframe = dataframe.copy()
    dataframe = dataframe[dataframe['Target'] != 'None']

    eye_to_object = dataframe[['PosX', 'PosY', 'PosZ']].values - dataframe[['PlayerPosX', 'PlayerPosY', 'PlayerPosZ']].values
    straight_ahead = np.array([0, 0, 1])  # This assumes that straight ahead is along the z-axis

    # Normalize the vectors (make them length 1) so that dot product gives the cosine of the angle
    eye_to_object = eye_to_object / np.linalg.norm(eye_to_object, axis=1, keepdims=True)
    straight_ahead = straight_ahead / np.linalg.norm(straight_ahead)

    # Calculate the dot product of the two vectors, which is equal to the cosine of the angle between them
    cos_angle = np.dot(eye_to_object, straight_ahead)

    # Calculate the angle itself (in radians)
    angle = np.arccos(cos_angle)

    # Convert to degrees
    angle = np.degrees(angle)

    # Normalize the angle by the Oculus Quest 2's field of view
    fov = 89
    angle = angle / fov

    # Add the angle to the dataframe
    dataframe['Eccentricity'] = angle

    return dataframe

calculate_eccentricity(dataframes[0])


,GazeTime,Region,Target,PosX,PosY,PosZ,PlayerPosX,PlayerPosY,PlayerPosZ,LightIntensity,Velocity,Eccentricity
0,0.166807,1,Size(Max),0.441291,0.450383,0.898136,0.000000,0.569430,0.000000,NaN,NaN,0.303054
1,0.208184,1,Size(Max),0.501019,0.160109,1.000000,0.000000,0.504816,0.113474,NaN,NaN,0.387075
2,0.250302,1,Size(Max),0.584421,0.085556,0.985626,0.000000,0.427128,0.249906,NaN,NaN,0.478836
3,0.306064,1,Size(Max),0.612888,0.000000,0.969624,0.000000,0.332133,0.416732,NaN,NaN,0.579559
4,0.097383,1,Size(Max),0.570646,0.520018,0.787474,0.000000,0.301907,0.469812,NaN,NaN,0.702543
5,0.097494,1,Size(Max),0.692646,0.038718,0.964657,0.000000,0.271646,0.522953,NaN,NaN,0.661230
6,0.110868,1,Size(Max),0.755920,0.051888,0.960085,0.000000,0.237235,0.583383,NaN,NaN,0.721046
19,0.180809,2,Transparency(Min),0.790352,0.504473,0.126212,0.497533,0.986739,0.462303,NaN,NaN,1.357100
20,0.222286,2,Transparency(Min),0.851023,0.490412,0.134069,0.498150,0.988439,0.341128,NaN,NaN,1.221784
21,0.194170,2,Transparency(Min),0.904499,0.480939,0.140959,0.498690,0.989924,0.235280,NaN,NaN,1.103871
